In [3]:
import glob, os, sys; sys.path.append('../src')

from typing import Callable, Dict, List, Optional

import pandas as pd
from pathlib import Path
import re
import logging
import string 
import pandas as pd
#from keybert import KeyBERT

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

logger = logging.getLogger(__name__)
import haystack
from haystack.utils import convert_files_to_docs, fetch_archive_from_http
from haystack.nodes.file_converter import BaseConverter, DocxToTextConverter, PDFToTextConverter, TextConverter
from haystack.schema import Document
import pdfplumber

from haystack.nodes import PreProcessor
import streamlit as st

In [4]:
# Function for reading txt,pdf and docx files 

def load_document(
    file: str,
    encoding: Optional[str] = None,
    id_hash_keys: Optional[List[str]] = None,
) -> List[Document]:
    
    """
    takes docx, txt and pdf files as input and extracts text as well as the filename as metadata. Since haystack
    does not take care of all pdf files, pdfplumber is attached to the pipeline in case the pdf extraction fails
    via Haystack.

    Returns a list of type haystack.schema.Document
    """

    if file.endswith('.pdf'):
        converter = PDFToTextConverter(remove_numeric_tables=True)
    if file.endswith('.txt'):
        converter = TextConverter()
    if file.endswith('.docx'):
        converter = DocxToTextConverter()

    print(converter)
    documents = []

    logger.info("Converting {}".format(file))
    # PDFToTextConverter, TextConverter, and DocxToTextConverter return a list containing a single Document
    document = converter.convert(
                file_path=file, meta=None, encoding=encoding, id_hash_keys=id_hash_keys
            )[0]
    text = document.content
    documents.append(Document(content=text, meta={"name": file}, id_hash_keys=id_hash_keys))
    
    '''check if text is empty and apply different pdf processor. This can happen whith certain pdf types.'''
    
    return documents

In [5]:
'''basic cleaning - suitable for transformer models'''
def basic(s):
    """
    :param s: string to be processed
    :return: processed string: see comments in the source code for more info
    """
    # Text Lowercase
    s = s.lower() 
    # Remove punctuation
    translator = str.maketrans(' ', ' ', string.punctuation) 
    s = s.translate(translator)
    # Remove URLs
    s = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', s, flags=re.MULTILINE)
    s = re.sub(r"http\S+", " ", s)
    # Remove new line characters
    s = re.sub('\n', ' ', s) 
  
    # Remove distracting single quotes
    s = re.sub("\'", " ", s) 
    # Remove all remaining numbers and non alphanumeric characters
    s = re.sub(r'\d+', ' ', s) 
    s = re.sub(r'\W+', ' ', s)

    # define custom words to replace:
    #s = re.sub(r'strengthenedstakeholder', 'strengthened stakeholder', s)
    
    return s.strip()

 

def preprocessing(document):

    """
    takes in haystack document object and splits it into paragraphs and applies simple cleaning.

    Returns cleaned list of haystack document objects. One paragraph per object. Also returns pandas df and 
    list that contains all text joined together.
    """    

    preprocessor = PreProcessor(
        clean_empty_lines=True,
        clean_whitespace=True,
        clean_header_footer=True,
        split_by="word",
        split_length=120,
        split_respect_sentence_boundary=True,
        #split_overlap=5
    )
    for i in document:
        docs_processed = preprocessor.process([i])
        for item in docs_processed:
            item.content = basic(item.content)

    print("your document has been splitted to", len(docs_processed), "paragraphs")
    
    # create dataframe of text and list of all text
    df = pd.DataFrame(docs_processed)
    all_text = " ".join(df.content.to_list())
    par_list = df.content.to_list()

    return df

In [6]:
import os

# Change the current working directory
os.chdir('C:\\Users\\serva\\Downloads\\NDCs')

# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))


cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))


# Safe directory in a var
directory_in_str='C:\\Users\\serva\\Downloads\\NDCs'
directory = os.fsencode(directory_in_str)


Current working directory: C:\Users\serva\Downloads\NDCs
Files in 'C:\\Users\\serva\\Downloads\\NDCs': ['Australias NDC June 2022 Update.docx', 'BOTSWANA.docx', 'EU_NDC_Submission_December 2020.docx', 'Updated - First NDC - FINAL - PDF.docx']


In [7]:
#Test
""""data=pd.DataFrame(columns=["content","id","meta","score","embedding"])
df=preprocessing(docs)
data1=data.append(df)

print(data1)"""

data=pd.DataFrame(columns=["content","id","meta","score","embedding"])

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    docs=load_document(filename)
# Using the Preprocessor to create df and text 
    df = preprocessing(docs)
    data=data.append(df)


INFO - __main__ -  Converting Australias NDC June 2022 Update.docx
2022-08-16 18:06:04.401 INFO    __main__: Converting Australias NDC June 2022 Update.docx


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 77.41docs/s]

your document has been splitted to 14 paragraphs



C:\Users\serva\AppData\Local\Temp\ipykernel_1868\2648344278.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(df)
INFO - __main__ -  Converting BOTSWANA.docx
2022-08-16 18:06:05.886 INFO    __main__: Converting BOTSWANA.docx


100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 813.80docs/s]
C:\Users\serva\AppData\Local\Temp\ipykernel_1868\2648344278.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(df)
INFO - __main__ -  Converting EU_NDC_Submission_December 2020.docx
2022-08-16 18:06:05.930 INFO    __main__: Converting EU_NDC_Submission_December 2020.docx


...

  0%|                                                                                          | 0/1 [00:00<?, ?docs/s]WARNING - haystack.nodes.preprocessor.preprocessor -  One or more sentence found with word count higher than the split length.
2022-08-16 18:06:05.973 WARNING haystack.nodes.preprocessor.preprocessor: One or more sentence found with word count higher than the split length.
100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 191.72docs/s]
C:\Users\serva\AppData\Local\Temp\ipykernel_1868\2648344278.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(df)
INFO - __main__ -  Converting Updated - First NDC - FINAL - PDF.docx
2022-08-16 18:06:06.003 INFO    __main__: Converting Updated - First NDC - FINAL - PDF.docx


your document has been splitted to 38 paragraphs


  0%|                                                                                          | 0/1 [00:00<?, ?docs/s]WARNING - haystack.nodes.preprocessor.preprocessor -  One or more sentence found with word count higher than the split length.
2022-08-16 18:06:06.021 WARNING haystack.nodes.preprocessor.preprocessor: One or more sentence found with word count higher than the split length.
100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 146.92docs/s]

your document has been splitted to 40 paragraphs



C:\Users\serva\AppData\Local\Temp\ipykernel_1868\2648344278.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(df)


In [8]:
data.head(5)
len(data)

101

In [9]:
# Rename the columns 

data=df.rename(columns = {'content':'Text'})

data.head(10)

,Text,content_type,id,meta,score,embedding
0,federative republic of brazil paris agreement nationally determined contribu...,text,986c33e39b7ad3e80d26bb72742c73d6,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 0}",None,None
1,brazil s updated ndc is broad in scope and includes a consideration of means...,text,76065630447e731a0abdb92a88f14ea2,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 1}",None,None
2,annex information to facilitate clarity transparency and understanding of br...,text,7165ebf036a1aa030c7d5ce64244c15b,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 2}",None,None
3,brazil will adopt the latest national inventory report available and submitt...,text,ac64b0d84efa5f32553ac705532ba503,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 3}",None,None
4,information on sources of data used in quantifying the reference points nati...,text,d93c9fc0d4b66988120244c50593d662,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 4}",None,None
5,net emissions from to compared with net emissions from to whether it is a si...,text,f4192996eea8b1b3d7955372b5815946,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 5}",None,None
6,how the party has taken into consideration paragraph c and d of decision cp ...,text,ecbf8a1825265edc67e394a357c8c9f3,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 6}",None,None
7,according to the working group i contribution to the sixth assessment report...,text,783a824d994caeffb6b4e40378471545,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 7}",None,None
8,adaptation actions implemented in the context of this ndc will aim at reduci...,text,87727313fe09b33ffe8f651608b87c50,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 8}",None,None
9,adaptation policies will be based on the best available science regarding cl...,text,3a502ab088e2c33f7ce088babee719a,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 9}",None,None


In [10]:
data.columns = map(str.lower, data.columns)
data.head(6)

,text,content_type,id,meta,score,embedding
0,federative republic of brazil paris agreement nationally determined contribu...,text,986c33e39b7ad3e80d26bb72742c73d6,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 0}",None,None
1,brazil s updated ndc is broad in scope and includes a consideration of means...,text,76065630447e731a0abdb92a88f14ea2,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 1}",None,None
2,annex information to facilitate clarity transparency and understanding of br...,text,7165ebf036a1aa030c7d5ce64244c15b,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 2}",None,None
3,brazil will adopt the latest national inventory report available and submitt...,text,ac64b0d84efa5f32553ac705532ba503,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 3}",None,None
4,information on sources of data used in quantifying the reference points nati...,text,d93c9fc0d4b66988120244c50593d662,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 4}",None,None
5,net emissions from to compared with net emissions from to whether it is a si...,text,f4192996eea8b1b3d7955372b5815946,"{'name': 'Updated - First NDC - FINAL - PDF.docx', '_split_id': 5}",None,None


In [11]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [12]:

documents=data["text"]

documents.head(10)



0    federative republic of brazil paris agreement nationally determined contribu...
1    brazil s updated ndc is broad in scope and includes a consideration of means...
2    annex information to facilitate clarity transparency and understanding of br...
3    brazil will adopt the latest national inventory report available and submitt...
4    information on sources of data used in quantifying the reference points nati...
5    net emissions from to compared with net emissions from to whether it is a si...
6    how the party has taken into consideration paragraph c and d of decision cp ...
7    according to the working group i contribution to the sixth assessment report...
8    adaptation actions implemented in the context of this ndc will aim at reduci...
9    adaptation policies will be based on the best available science regarding cl...
Name: text, dtype: object

In [14]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [15]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [16]:

processed_docs = data['text'].map(preprocess)



In [17]:
processed_docs[:10]

0    [feder, republ, brazil, pari, agreement, nation, determin, contribut, brasíl...
1    [brazil, updat, broad, scope, includ, consider, mean, implement, implement, ...
2    [annex, inform, facilit, clariti, transpar, understand, brazil, quantifi, in...
3    [brazil, adopt, latest, nation, inventori, report, avail, submit, unfccc, ti...
4    [inform, sourc, data, quantifi, refer, point, nation, inventori, anthropogen...
5    [emiss, compar, emiss, singleyear, multiyear, target, applic, singleyear, ta...
6    [parti, take, consider, paragraph, decis, gas, previous, indic, indc, keep, ...
7    [accord, work, group, contribut, sixth, assess, report, ipcc, publish, augus...
8    [adapt, action, implement, context, reduc, vulner, term, water, energi, food...
9    [adapt, polici, base, best, avail, scienc, climat, chang, nation, circumst, ...
Name: text, dtype: object

# Bag of Words


In [18]:
dictionary = gensim.corpora.Dictionary(processed_docs)

INFO - gensim.corpora.dictionary -  adding document #0 to Dictionary(0 unique tokens: [])
2022-08-16 14:55:30.671 INFO    gensim.corpora.dictionary: adding document #0 to Dictionary(0 unique tokens: [])
INFO - gensim.corpora.dictionary -  built Dictionary(631 unique tokens: ['achiev', 'addit', 'adopt', 'agreement', 'brasília']...) from 40 documents (total 2198 corpus positions)
2022-08-16 14:55:30.676 INFO    gensim.corpora.dictionary: built Dictionary(631 unique tokens: ['achiev', 'addit', 'adopt', 'agreement', 'brasília']...) from 40 documents (total 2198 corpus positions)
INFO - gensim.utils -  Dictionary lifecycle event {'msg': "built Dictionary(631 unique tokens: ['achiev', 'addit', 'adopt', 'agreement', 'brasília']...) from 40 documents (total 2198 corpus positions)", 'datetime': '2022-08-16T14:55:30.677944', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'created'}
2022-08-16 14:5

In [19]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 achiev
1 addit
2 adopt
3 agreement
4 brasília
5 brazil
6 chang
7 climat
8 commit
9 communic
10 compar


In [20]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[10]

[(6, 1),
 (7, 1),
 (18, 1),
 (21, 1),
 (26, 1),
 (40, 2),
 (71, 1),
 (115, 3),
 (153, 1),
 (163, 2),
 (221, 1),
 (249, 1),
 (254, 1),
 (255, 1),
 (256, 1),
 (257, 1),
 (258, 1),
 (259, 1),
 (260, 1),
 (261, 1),
 (262, 1),
 (263, 1),
 (264, 1)]

In [21]:
bow_doc_4310 = bow_corpus[10]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))



Word 6 ("chang") appears 1 time.
Word 7 ("climat") appears 1 time.
Word 18 ("feder") appears 1 time.
Word 21 ("govern") appears 1 time.
Word 26 ("nation") appears 1 time.
Word 40 ("adapt") appears 2 time.
Word 71 ("sector") appears 1 time.
Word 115 ("plan") appears 3 time.
Word 153 ("polici") appears 1 time.
Word 163 ("effort") appears 2 time.
Word 221 ("resili") appears 1 time.
Word 249 ("scientif") appears 1 time.
Word 254 ("complement") appears 1 time.
Word 255 ("criteria") appears 1 time.
Word 256 ("entiti") appears 1 time.
Word 257 ("incorpor") appears 1 time.
Word 258 ("knowledg") appears 1 time.
Word 259 ("mainstream") appears 1 time.
Word 260 ("promot") appears 1 time.
Word 261 ("strateg") appears 1 time.
Word 262 ("subject") appears 1 time.
Word 263 ("subnat") appears 1 time.
Word 264 ("support") appears 1 time.


In [25]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

INFO - gensim.models.ldamodel -  using symmetric alpha at 0.1
2022-08-16 14:55:52.374 INFO    gensim.models.ldamodel: using symmetric alpha at 0.1
INFO - gensim.models.ldamodel -  using symmetric eta at 0.1
2022-08-16 14:55:52.376 INFO    gensim.models.ldamodel: using symmetric eta at 0.1
INFO - gensim.models.ldamodel -  using serial LDA version on this node
2022-08-16 14:55:52.377 INFO    gensim.models.ldamodel: using serial LDA version on this node
INFO - gensim.models.ldamulticore -  running online LDA training, 10 topics, 2 passes over the supplied corpus of 40 documents, updating every 4000 documents, evaluating every ~40 documents, iterating 50x with a convergence threshold of 0.001000
2022-08-16 14:55:52.385 INFO    gensim.models.ldamulticore: running online LDA training, 10 topics, 2 passes over the supplied corpus of 40 documents, updating every 4000 documents, evaluating every ~40 documents, iterating 50x with a convergence threshold of 0.001000
WARNING - gensim.models.ldamul

In [27]:
for index, score in sorted(lda_model[bow_corpus[12]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9859340786933899	 
Topic: 0.032*"refer" + 0.026*"approach" + 0.025*"nation" + 0.023*"includ" + 0.022*"inform" + 0.021*"brazil" + 0.020*"emiss" + 0.020*"applic" + 0.019*"contribut" + 0.019*"methodolog"


In [28]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

from pprint import pprint

import spacy

import pickle
import re 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import matplotlib.pyplot as plt 
import pandas as pd

In [29]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)
p

C:\Users\serva\Downloads\Anaconda\envs\py39\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.018309 -0.091823       1        1  19.802743
2     -0.168457  0.033137       2        1  17.498733
8      0.048320 -0.051488       3        1  15.732607
7      0.000249 -0.012834       4        1  12.778181
0      0.090930  0.124670       5        1   9.489100
9     -0.006630  0.034043       6        1   7.792639
1      0.014912 -0.022407       7        1   6.510746
3     -0.011273  0.011444       8        1   4.409722
5      0.076189 -0.047953       9        1   3.944646
4     -0.025931  0.023210      10        1   2.040883, topic_info=          Term       Freq      Total Category  logprob  loglift
92       refer  15.000000  15.000000  Default  30.0000  30.0000
104     articl  16.000000  16.000000  Default  29.0000  29.0000
26      nation  37.000000  37.000000  Default  28.0000  28.0000
5       brazil  34.000000  34.000000  Default  27.0000  27.0000
3    agreement  19.000000  19.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
14   contribut   0.635171  31.299267  Topic10  -4.2574  -0.0057
16    determin   0.489611  18.842658  Topic10  -4.5177   0.2415
115       plan   0.401951  18.856278  Topic10  -4.7149   0.0435
5       brazil   0.399110  34.583749  Topic10  -4.7220  -0.5701
31       parti   0.354741  20.444698  Topic10  -4.8399  -0.1623

[572 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
158       1  0.273442  accord
158       2  0.136721  accord
158       3  0.136721  accord
158       5  0.136721  accord
158       7  0.136721  accord
...     ...       ...     ...
532       9  0.242323   world
103       1  0.088007    year
103       2  0.616047    year
103       3  0.088007    year
103       6  0.176014    year

[803 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 3, 9, 8, 1, 10, 2, 4, 6, 5])

In [30]:
unseen_document = 'Determining this huge process was quiet nice'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

Score: 0.41601282358169556	 Topic: 0.009*"stronger" + 0.009*"contribut" + 0.007*"articl" + 0.006*"adapt" + 0.006*"resili"
Score: 0.3170984983444214	 Topic: 0.012*"transpar" + 0.009*"clariti" + 0.009*"understand" + 0.009*"decis" + 0.009*"refer"
Score: 0.03336329013109207	 Topic: 0.007*"updat" + 0.007*"ambit" + 0.007*"independ" + 0.007*"greenhous" + 0.006*"public"
Score: 0.03336196765303612	 Topic: 0.009*"industri" + 0.008*"train" + 0.008*"energi" + 0.008*"data" + 0.008*"resourc"
Score: 0.03336058184504509	 Topic: 0.003*"transpar" + 0.003*"contribut" + 0.003*"target" + 0.003*"understand" + 0.003*"facilit"
Score: 0.03336058184504509	 Topic: 0.003*"transpar" + 0.003*"refer" + 0.003*"target" + 0.003*"contribut" + 0.003*"understand"
Score: 0.03336057439446449	 Topic: 0.014*"report" + 0.010*"biennial" + 0.010*"detail" + 0.008*"transpar" + 0.007*"target"
Score: 0.03336057439446449	 Topic: 0.013*"vehicl" + 0.010*"electr" + 0.007*"agenc" + 0.006*"establish" + 0.006*"commonwealth"
Score: 0.033360